# 🧪 TPC-DS Benchmark Queries 36–40 (Fixed Query 37)
Databricks SQL using `%run ./specify_catalog_schema`

In [0]:
%run ./specify_catalog_schema

In [0]:
%sql
-- Query 36: Inventory Performance by Category and Class
-- TPC-DS Query 36
SELECT
    SUM(ss_net_profit) / SUM(ss_ext_sales_price) AS gross_margin,
    i_category,
    i_class,
    GROUPING(i_category) + GROUPING(i_class) AS lochierarchy,
    RANK() OVER (
        PARTITION BY GROUPING(i_category) + GROUPING(i_class),
        CASE WHEN GROUPING(i_class) = 0 THEN i_category END
        ORDER BY SUM(ss_net_profit) / SUM(ss_ext_sales_price) ASC
    ) AS rank_within_parent
FROM
    store_sales
    JOIN date_dim ON d_date_sk = ss_sold_date_sk
    JOIN item ON i_item_sk = ss_item_sk
    JOIN store ON s_store_sk = ss_store_sk
WHERE
    d_year = 2000
    AND s_state IN ('TN', 'TN', 'TN', 'TN', 'TN', 'TN', 'TN', 'TN')
GROUP BY
    ROLLUP(i_category, i_class)
ORDER BY
    lochierarchy DESC,
    CASE WHEN lochierarchy = 0 THEN i_category END,
    rank_within_parent
LIMIT 100;

In [0]:
%sql
-- Query 37: Inventory Analysis by Price and Manufacturer (Fixed)
-- TPC-DS Query 37 (Fixed for Databricks SQL)
SELECT
    i_item_id,
    i_item_desc,
    i_current_price
FROM
    item
    JOIN inventory ON inv_item_sk = i_item_sk
    JOIN date_dim ON d_date_sk = inv_date_sk
    JOIN catalog_sales ON cs_item_sk = i_item_sk
WHERE
    i_current_price BETWEEN 22 AND 52
    AND d_date BETWEEN DATE('2001-06-02') AND DATE_ADD(DATE('2001-06-02'), 60)
    AND i_manufact_id IN (678, 964, 918, 849)
    AND inv_quantity_on_hand BETWEEN 100 AND 500
GROUP BY
    i_item_id,
    i_item_desc,
    i_current_price
ORDER BY
    i_item_id
LIMIT 100;

In [0]:
%sql
-- Query 38: Customer Shopping Patterns Across Channels
-- TPC-DS Query 38
SELECT COUNT(*)
FROM (
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM store_sales
        JOIN date_dim ON ss_sold_date_sk = d_date_sk
        JOIN customer ON ss_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1212 AND 1223
    INTERSECT
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM catalog_sales
        JOIN date_dim ON cs_sold_date_sk = d_date_sk
        JOIN customer ON cs_bill_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1212 AND 1223
    INTERSECT
    SELECT DISTINCT c_last_name, c_first_name, d_date
    FROM web_sales
        JOIN date_dim ON ws_sold_date_sk = d_date_sk
        JOIN customer ON ws_bill_customer_sk = c_customer_sk
    WHERE d_month_seq BETWEEN 1212 AND 1223
) AS cross_channel_customers;

In [0]:
%sql
-- Query 39: Inventory Variance Analysis by Warehouse and Month
-- TPC-DS Query 39
WITH inv AS (
    SELECT
        w_warehouse_name,
        w_warehouse_sk,
        i_item_sk,
        d_moy,
        STDDEV_SAMP(inv_quantity_on_hand) AS stdev,
        AVG(inv_quantity_on_hand) AS mean,
        CASE
            WHEN AVG(inv_quantity_on_hand) = 0 THEN NULL
            ELSE STDDEV_SAMP(inv_quantity_on_hand) / AVG(inv_quantity_on_hand)
        END AS cov
    FROM
        inventory
        JOIN item ON inv_item_sk = i_item_sk
        JOIN warehouse ON inv_warehouse_sk = w_warehouse_sk
        JOIN date_dim ON inv_date_sk = d_date_sk
    WHERE
        d_year = 1998
    GROUP BY
        w_warehouse_name,
        w_warehouse_sk,
        i_item_sk,
        d_moy
)
SELECT
    w_warehouse_name,
    d_moy,
    MAX(CASE WHEN cov IS NULL THEN 0 ELSE 1 END) AS cov_flag,
    MAX(cov) AS max_cov,
    MAX(CASE WHEN cov IS NULL THEN 0 ELSE 1 END) * AVG(cov) AS avg_cov,
    MAX(cov) / AVG(cov) AS cov_ratio
FROM
    inv
GROUP BY
    w_warehouse_name,
    d_moy
ORDER BY
    w_warehouse_name,
    d_moy
LIMIT 100;

In [0]:
%sql
-- Query 40: Sales Impact Analysis Before and After a Specific Date
-- TPC-DS Query 40
SELECT
    w_state,
    i_item_id,
    SUM(CASE WHEN d_date < DATE('1998-04-08') THEN cs_sales_price - COALESCE(cr_refunded_cash, 0) ELSE 0 END) AS sales_before,
    SUM(CASE WHEN d_date >= DATE('1998-04-08') THEN cs_sales_price - COALESCE(cr_refunded_cash, 0) ELSE 0 END) AS sales_after
FROM
    catalog_sales
    LEFT JOIN catalog_returns ON cs_order_number = cr_order_number AND cs_item_sk = cr_item_sk
    JOIN warehouse ON cs_warehouse_sk = w_warehouse_sk
    JOIN item ON cs_item_sk = i_item_sk
    JOIN date_dim ON cs_sold_date_sk = d_date_sk
WHERE
    i_current_price BETWEEN 0.99 AND 1.49
GROUP BY
    w_state,
    i_item_id
ORDER BY
    w_state,
    i_item_id
LIMIT 100;